In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_density_weighted_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    # Initialize NearestNeighbors for density calculation
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(X_pool)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]

        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        # Calculate uncertainty: 1 - max(probability)
        uncertainties = 1 - np.max(probs_c, axis=1)  # shape: (n_samples_in_class,)

        # Calculate density: inverse of sum of distances to k-nearest neighbors
        distances = nn.kneighbors(X_c, n_neighbors=5, return_distance=True)[0]
        density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)

        # Combine uncertainty and density
        scores = uncertainties * density

        # Select top-k samples with highest scores
        sorted_idx = np.argsort(scores)[-n_select:]
        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = vstack(X_selected).toarray() if len(X_selected) > 0 else np.array([])
    y_selected = np.concatenate(y_selected) if len(y_selected) > 0 else np.array([])

    # Remaining data
    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=11)}")

    return X_rest, X_selected, y_rest, y_selected


# Example usage
X_pool = x_test_adv_0_1  # Your data
y_pool = y_label_0_1     # Your labels (0 to 10)

# Initial stratified sampling
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)
print(f"Initial labeled set size: {len(y_train_init)}")
print(f"Initial label distribution: {np.bincount(y_train_init, minlength=11)}")

# Train initial model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)


_, X_1, _, y_1 = classwise_density_weighted_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.01, random_state=42
)

Initial labeled set size: 102570
Initial label distribution: [75542   161   700  1743   377 10542    74   736 12435   172    88]


/tmp/ipykernel_577/3022580865.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_577/3022580865.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_577/3022580865.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_577/3022580865.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_577/3022580865.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_577/3022580865.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_577/3022580865.py:59: RuntimeWarning: divide by zero encountered in divide
  

Selected samples (fraction=0.01): 10253
Label distribution: [7554   16   70  174   37 1054    7   73 1243   17    8]


In [6]:
X_1.shape

(10253, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt1 = DecisionTreeClassifier()
dt1.fit(X_1, y_1)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  8  9 10] [ 2449    64   116 16358   729   572 49099 44074   510]
(2449, 56) (2449,)
Save 0 to baseline_Def1.npy
(64, 56) (64,)
Save 1 to baseline_Def2.npy
(116, 56) (116,)
Save 2 to baseline_Def3.npy
(16358, 56) (16358,)
Save 3 to baseline_Def4.npy
(729, 56) (729,)
Save 4 to baseline_Def5.npy
(572, 56) (572,)
Save 5 to baseline_Def6.npy
(49099, 56) (49099,)
Save 8 to baseline_Def9.npy
(44074, 56) (44074,)
Save 9 to baseline_Def10.npy
(510, 56) (510,)
Save 10 to baseline_Def11.npy
Execution Time: 1.538141 seconds
(341913,)
[ 0  1  2  3  4  5  8  9 10] [147313    178    157  38526   4409  81320  23598  44709   1703]
(147313, 56) (147313,)
Save 0 to BIM_Def1.npy
(178, 56) (178,)
Save 1 to BIM_Def2.npy
(157, 56) (157,)
Save 2 to BIM_Def3.npy
(38526, 56) (38526,)
Save 3 to BIM_Def4.npy
(4409, 56) (4409,)
Save 4 to BIM_Def5.npy
(81320, 56) (81320,)
Save 5 to BIM_Def6.npy
(23598, 56) (23598,)
Save 8 to BIM_Def9.npy
(44709, 56) (44709,)
Save 9 to BIM_Def10.npy
(17

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  8  9 10] [ 4504    44   114  9566  7161   451 49573 41776   782]
(4504, 56) (4504,)
Save 0 to baseline_Def1.npy
(44, 56) (44,)
Save 1 to baseline_Def2.npy
(114, 56) (114,)
Save 2 to baseline_Def3.npy
(9566, 56) (9566,)
Save 3 to baseline_Def4.npy
(7161, 56) (7161,)
Save 4 to baseline_Def5.npy
(451, 56) (451,)
Save 5 to baseline_Def6.npy
(49573, 56) (49573,)
Save 8 to baseline_Def9.npy
(41776, 56) (41776,)
Save 9 to baseline_Def10.npy
(782, 56) (782,)
Save 10 to baseline_Def11.npy
Execution Time: 0.674461 seconds
(341913,)
[ 0  1  3  4  5  8  9 10] [185710    252  17607   7135  98435   8019  24445    310]
(185710, 56) (185710,)
Save 0 to BIM_Def1.npy
(252, 56) (252,)
Save 1 to BIM_Def2.npy
(17607, 56) (17607,)
Save 3 to BIM_Def4.npy
(7135, 56) (7135,)
Save 4 to BIM_Def5.npy
(98435, 56) (98435,)
Save 5 to BIM_Def6.npy
(8019, 56) (8019,)
Save 8 to BIM_Def9.npy
(24445, 56) (24445,)
Save 9 to BIM_Def10.npy
(310, 56) (310,)
Save 10 to BIM_Def11.npy
Execution Tim

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  8  9 10] [15975    95  2717 11018   503 10068 49937 22126  1532]
(15975, 56) (15975,)
Save 0 to baseline_Def1.npy
(95, 56) (95,)
Save 1 to baseline_Def2.npy
(2717, 56) (2717,)
Save 2 to baseline_Def3.npy
(11018, 56) (11018,)
Save 3 to baseline_Def4.npy
(503, 56) (503,)
Save 4 to baseline_Def5.npy
(10068, 56) (10068,)
Save 5 to baseline_Def6.npy
(49937, 56) (49937,)
Save 8 to baseline_Def9.npy
(22126, 56) (22126,)
Save 9 to baseline_Def10.npy
(1532, 56) (1532,)
Save 10 to baseline_Def11.npy
Execution Time: 0.203686 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [110659    290   4933  23207   2646 114463   6018  50779  26717   2201]
(110659, 56) (110659,)
Save 0 to BIM_Def1.npy
(290, 56) (290,)
Save 1 to BIM_Def2.npy
(4933, 56) (4933,)
Save 2 to BIM_Def3.npy
(23207, 56) (23207,)
Save 3 to BIM_Def4.npy
(2646, 56) (2646,)
Save 4 to BIM_Def5.npy
(114463, 56) (114463,)
Save 5 to BIM_Def6.npy
(6018, 56) (6018,)
Save 7 to BIM_Def8.npy
(50779, 56) (50779,)
Save 

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_DensityWeighted/UNSW_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_ActiveLearning_DensityWeighted.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,2449,64,116,16358,729,572,0,0,49099,44074,510,113971
1,XGB1,147313,178,157,38526,4409,81320,0,0,23598,44709,1703,341913
2,XGB1,93039,93,145,42825,4158,131313,0,0,23886,44863,1591,341913
3,XGB1,147313,178,157,38526,4409,81320,0,0,23598,44709,1703,341913
4,XGB1,68995,30,1647,41554,3540,177491,198,13144,272,33601,1441,341913
5,XGB1,130071,618,158,39369,2796,82803,5,0,30822,54112,1159,341913
6,XGB1,7671,217,354,49247,2374,1764,0,0,146327,132135,1824,341913
7,XGB1,45116,190,100,69181,2568,2921,0,0,129699,89843,2295,341913
8,XGB1,154679,1915,145,35582,6324,55165,0,0,23739,62107,2257,341913
9,XGB1,152243,286,145,36216,4962,78212,0,0,21584,46314,1951,341913
